<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/stock-etc/03-02-get-corporate-open-information-from-dart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DART 사이트에서 (삼성전자: 005930) 기업 공시 정보 가져오기 
1999-01-01부터 현재까지 분기/반기 보고서 가져와서 로컬 디렉토리에 csv로 저장하기

In [25]:
import requests
from io import BytesIO
import pandas as pd

In [27]:
crtfc_key = '94f6f704a87d5868d411db6c48a4bc01194d4bd2'
# crtfc_key = ""
# with open('./sample_data/my_data.txt', 'r') as reader:
#   crtfc_key = reader.readline()
  # print(crtfc_key)

In [44]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

In [28]:
from datetime import datetime
now = datetime.today().strftime('%Y%m%d')

In [29]:
corp_code = '005930' # 삼성전자

In [40]:
def make_api_url(corp_code, bgn_de='19990101', end_de=now, pblntf_ty='A', pblntf_detail_ty=['A001','A002','A003'], page_count='100'):

  for i, code in enumerate(pblntf_detail_ty):
    pblntf_detail_ty[i] = 'pblntf_detail_ty=' + code
    if i != (len(pblntf_detail_ty)-1): pblntf_detail_ty[i] += '&'
  pblntf_detail_ty = "".join(pblntf_detail_ty)

  url = "https://opendart.fss.or.kr/api/list.json?crtfc_key={crtfc_key}&corp_code={corp_code}&bgn_de={bgn_de}&end_de={end_de}&pblntf_ty={pblntf_ty}&{pblntf_detail_ty}&page_count={page_count}".format(\
                                                              crtfc_key=crtfc_key,corp_code=corp_code,bgn_de=bgn_de,end_de=end_de,pblntf_ty=pblntf_ty,pblntf_detail_ty=pblntf_detail_ty,page_count=page_count)
  return url

In [41]:
url = make_api_url(corp_code)
print(url)

https://opendart.fss.or.kr/api/list.json?crtfc_key=94f6f704a87d5868d411db6c48a4bc01194d4bd2&corp_code=005930&bgn_de=19990101&end_de=20210610&pblntf_ty=A&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003&page_count=100


In [45]:
response = requests.get(url, headers={"user-agent":user_agent})

In [70]:
pd.read_json(response.content)['list'][:-1]

0     {'corp_code': '00126380', 'corp_name': '삼성전자',...
1     {'corp_code': '00126380', 'corp_name': '삼성전자',...
2     {'corp_code': '00126380', 'corp_name': '삼성전자',...
3     {'corp_code': '00126380', 'corp_name': '삼성전자',...
4     {'corp_code': '00126380', 'corp_name': '삼성전자',...
                            ...                        
87    {'corp_code': '00126380', 'corp_name': '삼성전자',...
88    {'corp_code': '00126380', 'corp_name': '삼성전자',...
89    {'corp_code': '00126380', 'corp_name': '삼성전자',...
90    {'corp_code': '00126380', 'corp_name': '삼성전자',...
91    {'corp_code': '00126380', 'corp_name': '삼성전자',...
Name: list, Length: 92, dtype: object

In [47]:
# Byte raw data => BytesIO Object => Read Excel
f = BytesIO(response.content)
f

In [ ]:
sheets = ["연결 재무상태표","연결 손익계산서","연결 포괄손익계산서","연결 자본변동표","연결 현금흐름표","재무상태표","손익계산서"]

In [ ]:
# 엑셀 파일에서 sheet의 내용을 읽어 sheet 이름으로 파일에 저장
for sheet in sheets:
  df = pd.read_excel(f, sheet_name=sheet, skiprows=5)
  df.to_csv(sheet+".csv", encoding="euc-kr")


total 16
drwxr-xr-x 1 root root 4096 Jun  1 13:40 .
drwxr-xr-x 1 root root 4096 Jun  9 06:40 ..
drwxr-xr-x 4 root root 4096 Jun  1 13:40 .config
drwxr-xr-x 1 root root 4096 Jun  1 13:40 sample_data


In [ ]:
ls -al sample_data/

total 55512
drwxr-xr-x 1 root root     4096 Jun  1 13:40 ./
drwxr-xr-x 1 root root     4096 Jun  1 13:40 ../
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json*
-rw-r--r-- 1 root root   301141 Jun  1 13:40 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Jun  1 13:40 california_housing_train.csv
-rw-r--r-- 1 root root 18289443 Jun  1 13:40 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Jun  1 13:40 mnist_train_small.csv
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md*


In [ ]:
with open('./sample_data/my_data.txt', 'w') as f:
    f.writelines("94f6f704a87d5868d411db6c48a4bc01194d4bd2")

In [ ]:
ls -al sample_data/

total 55520
drwxr-xr-x 1 root root     4096 Jun  9 06:55 ./
drwxr-xr-x 1 root root     4096 Jun  1 13:40 ../
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json*
-rw-r--r-- 1 root root   301141 Jun  1 13:40 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Jun  1 13:40 california_housing_train.csv
-rw-r--r-- 1 root root 18289443 Jun  1 13:40 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Jun  1 13:40 mnist_train_small.csv
-rw-r--r-- 1 root root       40 Jun  9 07:01 my_data.txt
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md*


In [ ]:
with open('./sample_data/my_data.txt', 'r') as reader:
  # Read & print the entire file
  crte_data = reader.readline()
  print(crte_data)

94f6f704a87d5868d411db6c48a4bc01194d4bd2


In [ ]:
!cat ./sample_data/my_data.txt

94f6f704a87d5868d411db6c48a4bc01194d4bd2